In [93]:
import os
import numpy as np
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon
import pandas as pd
import pdb
from scrape_stores import scrape_stores_URL
import matplotlib.pyplot as plt
import time
import sys

In [16]:
#testing scraping function
URL = "https://www.starbucks.com/store-locator?map=38.898935,-77.026836,15z"

store_df = scrape_stores_URL(URL, [])
store_df.head()

,storeNumber,city,countrySubdivisionCode,countryCode,addressLines,hours,latitude,longitude
0,23466-230290,Washington,DC,US,1000 H Street NW,4:30 AM to 7:00 PM,38.89964,-77.02678
1,70004-26515,Washington,DC,US,775 12th St. NW,6:00 AM to 5:00 PM,38.89912,-77.02774
2,7784-66360,Washington,DC,US,"555 11th Street, NW",5:00 AM to 7:00 PM,38.89662,-77.02680
3,10725-100383,Washington,DC,US,901 New York Avenue NW,4:30 AM to 7:00 PM,38.90181,-77.02559
4,7569-238095,Washington,DC,US,1304 F Street NW,5:30 AM to 6:00 PM,38.89717,-77.03029


In [102]:
#Building a grid of points to exhaustively sample within a city
#lat, long steps at zoom = 15 ~ 2/3 as wide as area shown
long_step = 0.03
lat_step = 0.01

#reading in map of Washington DC
DC_path = os.path.join(r'C:\Users\Mehrab\test\DS_projects\Geo_starbucks_proj\Shapefile_data\Washington_DC_Boundary.shx')
DC_map = gpd.read_file(DC_path) 
DC_bound_box = DC_map.bounds #min-max lat and long values for city

#systematically sampling points within DC city limits to query store locator website
curr_pt_plot = 0
store_gdf = []
for curr_lat in np.arange(DC_bound_box.iloc[0, 1], DC_bound_box.iloc[0, 3], lat_step):
    for curr_long in np.arange(DC_bound_box.iloc[0, 0], DC_bound_box.iloc[0, 2], long_step):
        curr_pt = gpd.GeoDataFrame(geometry = gpd.points_from_xy([curr_long], [curr_lat]))
        if DC_map.contains(curr_pt.geometry).all():
            
            #logging scraped points
            if curr_pt_plot == 0:
                scraped_points = curr_pt
            else:
                scraped_points = pd.concat([scraped_points, curr_pt], ignore_index = False)
                
            #scraping store data from store locator website in region around curr_pt
            curr_URL = "https://www.starbucks.com/store-locator?map=" + str(round(curr_lat, 6)) + "," + str(round(curr_long, 6)) + ",15z"
            store_gdf = scrape_stores_URL(curr_URL, store_gdf)
            
            curr_pt_plot += 1
            print(curr_pt_plot)
            print(curr_URL)
            #pausing for 5s after every 10 queries to store locator website
            if curr_pt_plot%12 == 0:
                sys.exit()
                #time.sleep(5)
            
            
        #PICK UP THREAD HERE
        #fix non-unique store addition
        #filter out stores not within DC city limits
        #generate plot of sampled points, store locations on DC map (helper lines below)
        
        
        #plotting lines - CODE ME
        #ax2 = americas.plot(color = 'whitesmoke', edgecolor = 'black', figsize = (12, 12), linestyle = ':')
        #path_gdf.plot(color = 'blue', ax = ax2)
            


1
https://www.starbucks.com/store-locator?map=38.801644,-77.029795,15z
2
https://www.starbucks.com/store-locator?map=38.811644,-77.029795,15z
3
https://www.starbucks.com/store-locator?map=38.821644,-77.029795,15z
4
https://www.starbucks.com/store-locator?map=38.831644,-77.029795,15z
5
https://www.starbucks.com/store-locator?map=38.831644,-76.999795,15z
6
https://www.starbucks.com/store-locator?map=38.841644,-77.029795,15z
7
https://www.starbucks.com/store-locator?map=38.841644,-76.999795,15z
8
https://www.starbucks.com/store-locator?map=38.851644,-77.029795,15z
9
https://www.starbucks.com/store-locator?map=38.851644,-76.999795,15z
10
https://www.starbucks.com/store-locator?map=38.851644,-76.969795,15z
11
https://www.starbucks.com/store-locator?map=38.861644,-77.029795,15z
12
https://www.starbucks.com/store-locator?map=38.861644,-76.999795,15z


SystemExit: 

C:\Users\Mehrab\anaconda3\envs\GeoProjs\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [103]:
len(store_gdf['storeNumber'].unique())

50

In [104]:
store_gdf

,storeNumber,city,countrySubdivisionCode,countryCode,addressLines,hours,latitude,longitude
0,55104-288492,Washington,DC,US,2228 Martin Luther King Ave SE,5:00 AM to 6:30 PM,38.86366,-76.99040
1,16039-167106,Washington,DC,US,401 M St SE,6:00 AM to 7:00 PM,38.87619,-76.99980
2,10898-102107,Washington,DC,US,"1200 New Jersey Ave., SE",4:30 AM to 7:00 PM,38.87620,-77.00360
3,8076-150985,Washington,DC,US,1100 4th ST SW,6:00 AM to 8:00 PM,38.87748,-77.01692
4,7892-52479,Washington,DC,US,401 8th Street SE,5:00 AM to 7:00 PM,38.88365,-76.99515
...,...,...,...,...,...,...,...,...
95,7845-71781,Washington,DC,US,"1429 P Street, NW",5:00 AM to 7:30 PM,38.90978,-77.03302
96,21772-214865,Washington,DC,US,2001 L St NW,5:00 AM to 6:00 PM,38.90400,-77.04520
97,17055-159609,Arlington,VA,US,2925 S Glebe Rd,5:00 AM to 8:30 PM,38.84556,-77.06805
98,75238-85016,Washington,DC,US,900 23rd St NW,12:00 AM to 12:00 AM,38.90104,-77.05043


In [88]:
10%10

0

In [91]:
curr_pt_plot%24 == 0

True